In [ ]:
# 导入包
import pandas as pd
import time
import re
import json
from selenium import webdriver

# 打开Chrome（需配置webdriver）
browser = webdriver.Chrome()

def get_mgtv_danmu(month_num, day_num, num1, num2):
    step = 1
    df_all = pd.DataFrame()

    while True:
        try:
            # 第一集URL
            danmu_url = 'https://bullet-ws.hitv.com/bullet/2020/{}/{}/{}/{}/{}.json'.format(month_num, day_num, num1, num2, step)

            # 打印进度
            print('正在获取第{}页的信息'.format(step))
            step += 1
            # 获取弹幕
            browser.get(danmu_url)

            # 休眠3秒
            time.sleep(3)

            # 提取数据
            pattern1 = re.compile(r'<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">')
            pattern2 = re.compile(r'</pre></body></html>')

            data1 = re.sub(pattern1, '', browser.page_source)
            data2 = re.sub(pattern2, '', data1)

            # 解析数据
            js_data = json.loads(data2)

            # 获取数据
            all_data = js_data['data']['items']

            # id
            danmu_id = [i.get('id') for i in all_data]
            # uname
            uname = [i.get('uname') for i in all_data]
            # 内容
            content = [i.get('content') for i in all_data]
            # 时间
            danmu_time = [i.get('time') for i in all_data]
            # 点赞
            up_count = [i.get('v2_up_count') for i in all_data]
            # 分钟
            danmu_minites = step-1

            # 保存数据
            df_one = pd.DataFrame({
                'danmu_id': danmu_id,
                'uname': uname,
                'content': content,
                'danmu_time': danmu_time,
                'up_count': up_count,
                'danmu_minites': danmu_minites
            })

            # 循环追加
            df_all = df_all.append(df_one, ignore_index=True)

        except Exception as e:
            print(e)
            print('没有此页面, 爬虫结束')
            break

    return df_all

if __name__ == '__main__':
    # 
    df_1 = get_mgtv_danmu(month_num='06', day_num='21', num1=104556, num2=8337559) 